In [650]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re
import numpy as np

In [651]:
data = pd.read_html("/content/Выгрузка 9 апреля.xls")
df = data[1].drop(['№ п/п', '№документа', 'Исх. №Дата', 'Примечание'], axis=1)
df.head()

,Датарегистрации,Адресат,Автор,Краткоесодержание
0,09.04.2019,Уразов Р.Н.,Антонова С.В. (АО «Интерфакс»),Вх - О возможности использования сетевого изда...
1,09.04.2019,Уразов Р.Н.,Картошкин С.А. (Министерство просвещения Росси...,Вх - Об участии в совещании по вопросу доработ...
2,09.04.2019,Уразов Р.Н.,Цивилев С.Е. (Администрация Кемеровской области),Вх - О командировании Саликовой К. и Глушко Д....
3,09.04.2019,Иванюк Л.А.,Волков Г.А. (Министерство транспорта Российско...,Вх - Об участии в Чемпионате мира по профессио...
4,09.04.2019,Уразов Р.Н.,Атанов И.В. (Ставропольский государственный аг...,Вх - Об участии в церемонии открытия Вузовског...


In [652]:
df.isna().sum()

Датарегистрации      2
Адресат              0
Автор                0
Краткоесодержание    0
dtype: int64

In [653]:
df = df.fillna(-1)

In [654]:
df.isna().sum()

Датарегистрации      0
Адресат              0
Автор                0
Краткоесодержание    0
dtype: int64

In [655]:
df['Агенство'] = df.Автор.apply(lambda x: x.split("(")[1].replace(')',''))

In [656]:
df['Автор'] = df.Автор.apply(lambda x: x.split("(")[0] + x.split(")")[1])

In [657]:
agents = []
agents_number_one = ['Министерств', 'Правительств', 'Федеральн', 'Администр']
for i in df.Агенство:
  br = 0
  for t in agents_number_one:
    if br:
      break
    if t in i:
      br=1
      agents.append(1)
  if not(br):
    agents.append(0)

In [658]:
df['Тип_агенства'] = list(agents)
df.head()

,Датарегистрации,Адресат,Автор,Краткоесодержание,Агенство,Тип_агенства
0,09.04.2019,Уразов Р.Н.,Антонова С.В.,Вх - О возможности использования сетевого изда...,АО «Интерфакс»,0
1,09.04.2019,Уразов Р.Н.,Картошкин С.А.,Вх - Об участии в совещании по вопросу доработ...,Министерство просвещения Российской Федерации,1
2,09.04.2019,Уразов Р.Н.,Цивилев С.Е.,Вх - О командировании Саликовой К. и Глушко Д....,Администрация Кемеровской области,1
3,09.04.2019,Иванюк Л.А.,Волков Г.А.,Вх - Об участии в Чемпионате мира по профессио...,Министерство транспорта Российской Федерации,1
4,09.04.2019,Уразов Р.Н.,Атанов И.В.,Вх - Об участии в церемонии открытия Вузовског...,Ставропольский государственный аграрный универ...,0


In [659]:
!pip install catboost
!pip install lightgbm
!pip install -q pydot

In [660]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier, plot_importance, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

In [720]:
x = df.Агенство
y = df.Адресат

In [721]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = 1000)

In [722]:
vectorizer = TfidfVectorizer(max_features = 100)

In [723]:
X_train = vectorizer.fit_transform(x_train)
X_test = vectorizer.transform(x_test)

In [714]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, f1_score

In [737]:
%%time
model_SVC = SVC(C = 1, kernel = 'linear')
model_SVC.fit(X_train, y_train)
res = model_SVC.predict(X_test)

CPU times: user 1.53 s, sys: 7.87 ms, total: 1.53 s
Wall time: 1.54 s


Точность - 0.02 Плохой резульат

In [738]:
f1_svm = f1_score(y_test, res, average='macro')
f1_svm

0.029414227977803676

In [739]:
print(classification_report(y_test, res))

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 precision    recall  f1-score   support

                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Абдуллин А.М.       0.00      0.00      0.00

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **RandomForestClassifier**

In [670]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score

Точность 0.63 Лучший результат

In [758]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
display(rfc.score(X_train, y_train))

0.6369101292455666

In [741]:
rf_cls = RandomForestClassifier(n_estimators = 100)

In [742]:
%%time
rf_cls.fit(X_train, y_train)

CPU times: user 1.17 s, sys: 47.1 ms, total: 1.22 s
Wall time: 1.22 s


RandomForestClassifier()

In [743]:
y_pred = rf_cls.predict(X_test)

In [744]:
print(classification_report(y_test, y_pred))

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 precision    recall  f1-score   support

                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Абдуллин А.М.       0.00      0.00      0.00

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

## **XGBClassifier**

In [751]:
xgb_cls = XGBClassifier(n_estimators = 100, learning_rate = 0.3, n_jobs = -1, random_state = 20)

In [752]:
%%time
xgb_cls.fit(X_train, y_train)

CPU times: user 12.7 s, sys: 51.6 ms, total: 12.8 s
Wall time: 14.8 s


XGBClassifier(learning_rate=0.3, n_jobs=-1, objective='multi:softprob',
              random_state=20)

In [753]:
y_pred = xgb_cls.predict(X_test)

Точность 0.59 Хороший результат

In [755]:
display(xgb_cls.score(X_train, y_train))

0.591824466486324

In [754]:
print(classification_report(y_test, y_pred))

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 precision    recall  f1-score   support

                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Абдуллин А.М.       0.00      0.00      0.00

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

## **LGBMClassifier**

In [729]:
from lightgbm import LGBMClassifier, plot_tree, plot_importance 

In [732]:
lgbm_cls = LGBMClassifier(n_estimators = 100, learning_rate= 1, random_state=1)

In [733]:
%%time
lgbm_cls.fit(X_train, y_train)

CPU times: user 3.12 s, sys: 102 ms, total: 3.22 s
Wall time: 3.22 s


LGBMClassifier(learning_rate=1, random_state=1)

In [734]:
y_pred = lgbm_cls.predict(X_test)

0.05 Плохой результат

In [756]:
display(lgbm_cls.score(X_train, y_train))

0.05530507965133754

In [735]:
print(classification_report(y_test, y_pred))

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 precision    recall  f1-score   support

                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Абдуллин А.М.       0.00      0.00      0.00

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

## **GridSearchCV**

In [684]:
params = {"n_estimators": np.arange(100,300,100)}

In [685]:
rf_cls = RandomForestClassifier()

In [686]:
grid = GridSearchCV(rf_cls, params, n_jobs = -1)

In [687]:
%%time
grid.fit(X_train, y_train)

CPU times: user 857 ms, sys: 38.3 ms, total: 895 ms
Wall time: 9.64 s


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'n_estimators': array([100, 200])})

In [688]:
y_pred = grid.best_estimator_.predict(X_test)

In [689]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.75      0.71      1003
           1       0.52      0.43      0.47       637

    accuracy                           0.62      1640
   macro avg       0.60      0.59      0.59      1640
weighted avg       0.61      0.62      0.61      1640



## **RandomForestClassifier** показал лучшие результаты что по времени, что и по точности